### Displaying data from pandas dataframes

* Displaying tabular data from Pandas DataFrames in a Tkinter GUI presents certain challenges due to the limitations of basic Tkinter widgets when dealing with large datasets.
* While widgets like Label or a series of Entry widgets can be used for small datasets, they become inefficient and cumbersome for larger, more complex data structures.
* Therefore, more specialized approaches are needed to effectively display Pandas DataFrames in a Tkinter application.


One approach
- One straightforward method to display a Pandas DataFrame is by converting it to a string representation using the df.to_string() method and then inserting this string into a Text widget.
- The Text widget is designed to handle multi-line text, making it suitable for displaying tabular data.
- Additionally, it's possible to redirect the standard output (sys.stdout) to a Text widget, allowing the output of print(df) to be displayed directly in the GUI.

In [4]:

# Some extra info included. protocol for root (can be used for any window) , destroy() can be use to remove any widget or window. Toplevel can be used to create a new connected window.
import tkinter as tk
from tkinter import ttk
import pandas as pd
import sys

def on_closing():
    root.destroy()
    #sys.exit()


root = tk.Tk()
root.title("Displaying DataFrame")
root.protocol("WM_DELETE_WINDOW", on_closing)
'''The following protocols are available for use:
WM_DELETE_WINDOW: Handles window close requests (e.g., clicking the close button).
WM_TAKE_FOCUS: (Not fully supported in Tk) Used for handling focus requests.
_NET_WM_PING: (Handled internally by Tk) Used by window managers to check if the application is responsive.'''
new_window = tk.Toplevel(root)
new_window.title("New Window")


df = pd.read_csv("SimpleExample.csv")

text_area = tk.Text(root, height=10, width=60)
text_area.pack(padx=10, pady=10)

text_area.insert(tk.END, df.to_string())

root.mainloop()

More structured approach

* using the ttk.Treeview widget. Although primarily designed for hierarchical data, Treeview can effectively display tabular data with headers and rows.
* This method involves setting up the columns of the Treeview based on the DataFrame's column names and then iterating through the rows of the DataFrame to insert the data into the Treeview.
* The tree.heading() method is used to set the column headers, and tree.insert() adds the data rows, aligning values with the correct columns.

More information here : https://tkdocs.com/tutorial/tree.html


In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd

root = tk.Tk()
root.title("DataFrame in Treeview")

df = pd.read_csv("SimpleExample.csv")

tree = ttk.Treeview(root, columns=list(df.columns), show='headings')

for col in df.columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)

for index, row in df.iterrows():
    tree.insert("", tk.END, values=list(row))

tree.pack(padx=10, pady=10)

root.mainloop()

Like always , there might be a library for it.
* For a more feature-rich solution, the pandastable library provides a dedicated table widget specifically for displaying and interacting with Pandas DataFrames.
* After installing the library using pip install pandastable , the Table class from pandastable can be used to directly display a DataFrame with built-in functionalities such as sorting, filtering, cell editing, and more. The library also includes a DataExplore application for non-programmers to manipulate and view data.

more info: https://pandastable.readthedocs.io/en/latest/description.html


In [ ]:
#!pip install pandastable
import tkinter as tk
from pandastable import Table
import pandas as pd

root = tk.Tk()
root.title("DataFrame with pandastable")

df = pd.read_csv("SimpleExample.csv")
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

pt = Table(frame, dataframe=df)
pt.show()

root.mainloop()

### Embedding Matplotlib and Seaborn Plots

* Embedding Matplotlib plots in Tkinter involves using the matplotlib.backends.backend_tkagg module, which provides the necessary tools for integration.
* Key classes from this module include FigureCanvasTkAgg for drawing the Matplotlib figure onto a Tkinter canvas and NavigationToolbar2Tk for adding interactive plot controls.
* The process typically involves creating a Matplotlib Figure and adding subplots, plotting data on the subplot using standard Matplotlib functions, creating a FigureCanvasTkAgg object to embed the figure in the Tkinter window, and then using layout managers to place the canvas widget.
* Optionally, a NavigationToolbar2Tk can be added for interactive features like zooming and panning.

In [ ]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

root = tk.Tk()
root.title("Embedding Matplotlib Plot")

#All this is same as our matplotlib discussion
figure = plt.Figure(figsize=(6, 4), dpi=100)
subplot = figure.add_subplot(111)
subplot.plot([4, 5, 6, 7, 8], [8, 7, 9, 6, 10])
subplot.set_title("Sample Plot")

#This is the additional stuff
canvas = FigureCanvasTkAgg(figure, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

root.mainloop()

Some simple ways to make plots interactive

In [ ]:
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.widgets import Slider
import numpy as np
root = tk.Tk()
root.title("Slider Plot Example")

#create the figure
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.25) # Make space for the slider
x = np.arange(0, 10, 0.1)
y = np.sin(x)
line = ax.plot(x, y) # create a line object with x and y


ax_slider = plt.axes([0.2, 0.1, 0.65, 0.03]) # adds a axes (left, bottom, width, height)
#Slider is a widget from matplotlib not tkinter
slider = Slider(ax_slider, 'Frequency', 0.1, 10, valinit=1)

def update(val):
    freq = slider.val
    line.set_ydata(np.sin(freq * x))
    fig.canvas.draw_idle() # which "schedules a rendering the next time the GUI window is going to re-paint the screen"

slider.on_changed(update)
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

tk.mainloop()

Dynamic plots that update with new data



In [ ]:
# Option 1: Automatically/ time based.
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import random
root = tk.Tk()
root.title("Dynamic Plot")

fig, ax = plt.subplots()
line = ax.plot([], [])[0]  # Create an empty line object, note the [0], that is because, plot actually returns a list of plot objects/artists, we only need the first one which we want to update late.

#Embed the Matplotlib figure in Tkinter.
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack()


def update_plot(x_data, y_data):
    line.set_xdata(x_data)
    line.set_ydata(y_data)
    ax.relim()  # Recalculate limits
    ax.autoscale_view()  # Autoscale the view
    fig.canvas.draw_idle()  # Redraw the canvas smoothly


x_data = []
y_data = []
def add_data_point():
    x_data.append(len(x_data) + 1)
    y_data.append(random.randint(1, 10))
    update_plot(x_data, y_data)
    root.after(1000, add_data_point)  # Update every 1000 ms (1 second) # root.after() in Tkinter is a method used to schedule a function call after a specified delay in milliseconds.


add_data_point()
root.mainloop()

Option 2: Interaction based.

In [ ]:
# Option 1: Automatically/ time based.
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import random
root = tk.Tk()
root.title("Dynamic Plot")

fig, ax = plt.subplots()
line = ax.plot([], [])[0]  # Create an empty line object, note the [0], that is because, plot actually returns a list of plot objects/artists, we only need the first one which we want to update late.

#Embed the Matplotlib figure in Tkinter.
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack()


def update_plot(x_data, y_data):
    line.set_xdata(x_data)
    line.set_ydata(y_data)
    ax.relim()  # Recalculate limits
    ax.autoscale_view()  # Autoscale the view
    fig.canvas.draw_idle()  # Redraw the canvas smoothly


x_data = []
y_data = []


def update_on_button():
    # Add new data or modify existing data here
    x_data.append(len(x_data) + 1)
    y_data.append(random.randint(1, 10))
    update_plot(x_data, y_data)

update_button = tk.Button(root, text="Update Plot", command=update_on_button)
update_button.pack()

root.mainloop()

### Using these simple ideas above, you can make great dynamic displays of data.

### Talking to Databases - Connecting Tkinter

* Connecting GUI applications to databases is essential for persistent data storage and retrieval, enabling applications to manage dynamic information.
* The same steps as we discussed in database topic:
* A connection to a database file is established using sqlite3.connect('your_database.db'), which creates the file if it does not exist.
* A cursor object, created using conn.cursor(), allows the execution of SQL queries.
* Fundamental SQL commands such as CREATE TABLE, INSERT INTO, SELECT * FROM, and DELETE FROM can be executed using cursor.execute(), with placeholders (?) used to safely pass values and prevent SQL injection.
* Data is retrieved using methods like cursor.fetchone() or cursor.fetchall().
* It's crucial to save changes with conn.commit() and close the connection using conn.close().

Simple example with no ORM

In [ ]:
import tkinter as tk
from tkinter import ttk
import sqlite3

def create_table():
    conn = sqlite3.connect('mydatabase.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            age INTEGER
        )
    ''')
    conn.commit()
    conn.close()
    print("Table created!")

root = tk.Tk()
root.title("SQLite Example")

create_button = ttk.Button(root, text="Create Table", command=create_table)
create_button.pack(padx=10, pady=10)

root.mainloop()

In [ ]:
# example extended with some adding functionality

import tkinter as tk
from tkinter import ttk
import sqlite3

def add_user():
    name = name_entry.get()
    age = age_entry.get()
    conn = sqlite3.connect('mydatabase.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (name, age) VALUES (?,?)", (name, age))
    conn.commit()
    conn.close()
    name_entry.delete(0, tk.END)
    age_entry.delete(0, tk.END)
    print(f"User {name} added!")

root = tk.Tk()
root.title("Add User to SQLite")

ttk.Label(root, text="Name:").pack(padx=5, pady=5)
name_entry = ttk.Entry(root)
name_entry.pack(padx=5, pady=5)

ttk.Label(root, text="Age:").pack(padx=5, pady=5)
age_entry = ttk.Entry(root)
age_entry.pack(padx=5, pady=5)

add_button = ttk.Button(root, text="Add User", command=add_user)
add_button.pack(padx=10, pady=10)

root.mainloop()

### ORM Example
Refer back to the ORM discussion and resources for clarity on ORM

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    email = Column(String)

   # def __repr__(self): #dunder to compute official string representation of object
   #     return f"<User(name='{self.name}', age={self.age}, email='{self.email}')>"

engine = create_engine('sqlite:///mydatabase_orm.db')
Base.metadata.create_all(engine) # Creates the table in the database

#creating and binding the session.
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

#example of how to add a new user
new_user = User(name='Amith', age=36, email='belman')
session.add(new_user) #adding
session.commit() #committing


#example of how to query the user table.
users = session.query(User).all() #you can also use filter() to select particular entries.
for user in users:
    print(user.name, user.age, user.email) #without representation dunder
    #print(user)  # with representation dunder


session.close()

Take home practice exercise. Make a simple GUI to script to add and view users in the database. View the users in a separate window.